In [1]:
# pip install gym==0.21

In [2]:
import gym
import numpy as np

env = gym.make("ALE/Assault-v5", render_mode="human")
n_actions = env.action_space.n

gym.version.VERSION

'0.21.0'

In [3]:
def visualize(agent, t_max=10000):
    env_show = gym.make("ALE/Assault-v5", render_mode='human').env
    
    state = env_show.reset()[0]
    
    for t in range(t_max):
        env_show.render()
        
        #predict with agent
        
        action = np.random.choice(np.arange(n_actions))
        
        x = env_show.step(action)
        
        new_state = x[0]
        done = x[2]
        
        state = new_state
        if done: 
            break
    
    
    env_show.close()

In [5]:
visualize(0)